# Explore Paris

In [1]:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge BeautifulSoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline 

import json
from pandas.io.json import json_normalize
import requests
from geopy.geocoders import Nominatim
import folium
from bs4 import BeautifulSoup

### Figure out Paris 'boroughs'

From Wikipedia (https://en.wikipedia.org/wiki/Arrondissements_of_Paris) we know that Paris have 20 arronddissements, some kind like boroughs, but much smaller and hard to split into smaller units like neighborhood. So i will use arrondissement directly instead neighborhood. However this page do not includes coordinates. So we need to scape web to get them.

In [3]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Arrondissements_of_Paris")
df_raw=pd.DataFrame(tables[2])
df_raw

,"Arrondissement (R for Right Bank, L for Left Bank)",Name,Area (km2),Population(March 1999 census),Population(July 2005 estimate),Density (2005)(inhabitants per km2),Peak of population,Mayor
0,1st (Ie) R,Louvre,1.826 km2 (0.705 sq mi),16888,17700,9693,before 1861,Jean-François Legaret (LR)
1,2nd (IIe) R,Bourse,0.992 km2 (0.383 sq mi),19585,20700,20867,before 1861,Jacques Boutault (EELV)
2,3rd (IIIe) R,Temple,1.171 km2 (0.452 sq mi),34248,35100,29974,before 1861,Pierre Aidenbaum (PS)
3,4th (IVe) R,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),30675,28600,17864,before 1861,Ariel Weil (PS)
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),58849,60600,23849,1911,Florence Berthout (LR)
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),44919,45200,20984,1911,Jean-Pierre Lecoq (LR)
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),56985,55400,13552,1926,Rachida Dati (LR)
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),39314,38700,9972,1891,Jeanne d'Hauteserre (LR)
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),55838,58500,26847,1901,Delphine Bürkli (LR)
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),89612,88800,30705,1881,Alexandra Cordebard (PS)


In [4]:
pr_1 = pd.DataFrame(df_raw['Arrondissement (R for Right Bank, L for Left Bank)'].str.split(' ',2).tolist(),
                                   columns = ['Arrondissement1', 'Arrondissement2', 'Bank'])
pr_1['Name'] = df_raw['Name']
pr_1['Area (km2)'] = df_raw['Area (km2)']
#df_arr['Zip code'] = pd.Series(np.arange(75001, 750021))
df_area=pd.DataFrame(pr_1['Area (km2)'].str.split('km2',1).tolist())
df_area[0]=df_area[0].str.replace('\xa0','').astype(float).round(decimals=2) 
pr_1['Area (km2)']=df_area[0]
pr_1

,Arrondissement1,Arrondissement2,Bank,Name,Area (km2)
0,1st,(Ie),R,Louvre,1.83
1,2nd,(IIe),R,Bourse,0.99
2,3rd,(IIIe),R,Temple,1.17
3,4th,(IVe),R,Hôtel-de-Ville,1.60
4,5th,(Ve),L,Panthéon,2.54
5,6th,(VIe),L,Luxembourg,2.15
6,7th,(VIIe),L,Palais-Bourbon,4.09
7,8th,(VIIIe),R,Élysée,3.88
8,9th,(IXe),R,Opéra,2.18
9,10th,(Xe),R,Entrepôt,2.89


### Add coordinates

In this page http://zip-code.en.mapawi.com/france/1/arrondissement-de-paris/3/150/, we can get each arrondissement's coordinates by clicking on correspond button.

In [5]:
url = 'http://zip-code.en.mapawi.com/france/1/arrondissement-de-paris/3/150/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
sub_urls = soup.find('div', id='menu').find_all('a')
len(sub_urls)

21

In [6]:
df_geo=pd.DataFrame(columns=['Arr', 'Zip code', 'Latitude', 'Longitude'])

for url in sub_urls[1:]:
    href = url['href']
    req = requests.get(href)
    soup = BeautifulSoup(req.content, 'html.parser')
    data = soup.find('div', id='info').find_all('font', class_='data')
    df_geo=df_geo.append({'Arr': url.text, 'Zip code': data[0].text, 
                          'Latitude': data[-2].text, 
                          'Longitude': data[-1].text}, ignore_index=True)
df_geo['Latitude']=df_geo['Latitude'].astype(float)
df_geo['Longitude']=df_geo['Longitude'].astype(float)
df_geo.head()

,Arr,Zip code,Latitude,Longitude
0,Paris 01 Louvre,75001,48.8592,2.3417
1,Paris 02 Bourse,75002,48.8655,2.3426
2,Paris 03 Temple,75003,48.8637,2.3615
3,Paris 04 Hôtel-de-Ville,75004,48.8601,2.3507
4,Paris 05 Panthéon,75005,48.8448,2.3471


In [7]:
df_geo[['City','Arrondissement','Name']] = pd.DataFrame(df_geo['Arr'].str.split().tolist(), index= df_geo.index)
pr_2=df_geo[['Arrondissement', 'Name', 'Zip code', 'Latitude', 'Longitude']]
pr_2

,Arrondissement,Name,Zip code,Latitude,Longitude
0,01,Louvre,75001,48.8592,2.3417
1,02,Bourse,75002,48.8655,2.3426
2,03,Temple,75003,48.8637,2.3615
3,04,Hôtel-de-Ville,75004,48.8601,2.3507
4,05,Panthéon,75005,48.8448,2.3471
5,06,Luxembourg,75006,48.8493,2.3300
6,07,Palais-Bourbon,75007,48.8565,2.3210
7,08,Élysée,75008,48.8763,2.3183
8,09,Opéra,75009,48.8718,2.3399
9,10,Entrepôt,75010,48.8709,2.3561


In [8]:
df_paris=pr_2[:]
df_paris['Area (km2)']=pr_1['Area (km2)']
df_paris['Arrondissement']=pr_1['Arrondissement1']
df_paris

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Arrondissement,Name,Zip code,Latitude,Longitude,Area (km2)
0,1st,Louvre,75001,48.8592,2.3417,1.83
1,2nd,Bourse,75002,48.8655,2.3426,0.99
2,3rd,Temple,75003,48.8637,2.3615,1.17
3,4th,Hôtel-de-Ville,75004,48.8601,2.3507,1.60
4,5th,Panthéon,75005,48.8448,2.3471,2.54
5,6th,Luxembourg,75006,48.8493,2.3300,2.15
6,7th,Palais-Bourbon,75007,48.8565,2.3210,4.09
7,8th,Élysée,75008,48.8763,2.3183,3.88
8,9th,Opéra,75009,48.8718,2.3399,2.18
9,10th,Entrepôt,75010,48.8709,2.3561,2.89


Now we can see that only 12th and 16th arrondissements have areas apparently larger than others. From the map, we can tell that it is because of green land part. So, i will add another two coordinates to Paris data, 48.8624°N, 2.2492°E for Bois de Boulogne in 16th arrondissement, and 48.8283°N, 2.4330°E for Bois de Vicennne in 12th arrondissement.

In [9]:
df_paris=df_paris.append({'Arrondissement': '12th',
                 'Name': 'Bois-de-Vincenne',
                 'Zip code': '77012',
                 'Latitude': 48.8283,
                 'Longitude': 2.4330,
                 'Area (km2)': 9.95
                }, ignore_index=True)
df_paris=df_paris.append({'Arrondissement': '16th',
                 'Name': 'Bois-de-Boulogne',
                 'Zip code': '77016',
                 'Latitude': 48.8624,
                 'Longitude': 2.2492,
                 'Area (km2)': 8.46
                }, ignore_index=True)

In [10]:
df_paris.at[11, 'Area (km2)']=round(df_paris.at[11, 'Area (km2)']-9.95, 2)
df_paris.at[15, 'Area (km2)']=round(df_paris.at[15, 'Area (km2)']-8.46, 2)
df_paris

,Arrondissement,Name,Zip code,Latitude,Longitude,Area (km2)
0,1st,Louvre,75001,48.8592,2.3417,1.83
1,2nd,Bourse,75002,48.8655,2.3426,0.99
2,3rd,Temple,75003,48.8637,2.3615,1.17
3,4th,Hôtel-de-Ville,75004,48.8601,2.3507,1.60
4,5th,Panthéon,75005,48.8448,2.3471,2.54
5,6th,Luxembourg,75006,48.8493,2.3300,2.15
6,7th,Palais-Bourbon,75007,48.8565,2.3210,4.09
7,8th,Élysée,75008,48.8763,2.3183,3.88
8,9th,Opéra,75009,48.8718,2.3399,2.18
9,10th,Entrepôt,75010,48.8709,2.3561,2.89


In [11]:
df_paris.to_csv("paris_arrondissements.csv")

### Show Paris map

In [12]:
address = 'Paris, France'
geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_paris = folium.Map(location=[latitude, longitude], zoom_start=11)

In [13]:
for lat, lng, arr, name in zip(df_paris['Latitude'], df_paris['Longitude'], 
                                        df_paris['Arrondissement'], df_paris['Name']):
    label = '{} {}'.format(arr, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        #fill_color='#3186cc',
        #fill_opacity=0.7,
        parse_html=False).add_to(map_paris)    
map_paris